<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Images Global

Goal : have well trained models for image sampling

Data : celeba64,mnist,svhn,celeba32

Models : ConvLNPXL 

Loss : NLLLloss

Runs : 1

In [1]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

In [3]:

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [4]:
args = get_exp_args("exp_global", is_load=False)
len(args)

9

In [5]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24*1,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips"
                          )

In [6]:
jobs = executor.map_array(Run(), args)

In [7]:
############################################################

In [9]:
jobs

[SlurmJob<job_id=26835596_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_5, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_6, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_7, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26835596_8, task_id=0, state="COMPLETED">]

In [10]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-05-26 18:30:59,987) - Starting with JobEnvironment(job_id=26835596_0, hostname=learnfair0313, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-26 18:30:59,987) - Loading pickle: /private/home/yannd/projects/NPF/logs/26835596_0/26835596_0_submitted.pkl

--- Training mnist/ConvNPFXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1     -715.1263     -970.9311     +  262.5057
      2    -1032.0848    -1107.1719     +  262.1540
      3    -1069.1841    -1086.1381        262.4050
      4    -1113.1128    -1131.8428     +  262.6610
      5    -1129.7067    -1145.6990     +  262.6888
      6    -1122.8412    -1152.9933     +  262.7113
      7    -1146.4331    -1011.3204        262.6426
      8    -1160.8199    -1190.8976     +  262.7869
      9    -1164.0354    -1191.6066     +  262.6530
     10    -1167.6970    -1170.7042        262.4454
  

In [54]:
for j in jobs_32:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit ERROR (2020-05-24 20:41:35,817) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 186, in submitit_main
    process_job(args.folder)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 179, in process_job
    raise error
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submi

In [10]:
for job in jobs:
    job.cancel()